# BERT Pretraining for Hate Eficient Speech Clasification

> Indented block



In [ ]:
!pip install transformers
!pip install sentencepiece
!pip install ipywidgets
!pip install accelerate
!jupyter nbextension enable --py widgetsnbextension

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 24.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.1/311.1 kB 42.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 64.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 65.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 39.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 3.9 MB/s eta 0:00:00
Enabling notebook extension jupyter-js-widgets/extension...
Paths used for configuration of notebook: 
    	/root/.jupyter/nbconfig/notebook.json
Paths used for configuration of notebook: 
    	
      - Validating: OK
Paths used for configuration of notebook: 
    	/root/.jupyter/nbconfig/notebook.json


In [ ]:
import torch
import transformers
import torch.nn as nn
from torch.utils.data import DataLoader
from tqdm.notebook import tqdm

from transformers import Trainer, TrainingArguments
from transformers import BertTokenizer
from transformers import BertPreTrainedModel, BertModel

import pandas as pd
import numpy as np
import os

from sklearn.metrics import classification_report

if not torch.cuda.is_available():
  print('WARNING: You may want to change the runtime to GPU for faster training!')
  DEVICE = 'cpu'
else:
  DEVICE = 'cuda:0'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

%cd '/content/drive/MyDrive/Colab Notebooks/'
%mkdir './transformers'
%cd './transformers'

Mounted at /content/drive
/content/drive/MyDrive/Colab Notebooks
mkdir: cannot create directory ‘./transformers’: File exists
/content/drive/MyDrive/Colab Notebooks/transformers


In [ ]:
%mkdir ./data
%cd ./data

if not os.path.isfile('pretrain.txt'):
  !wget -O pretrain.txt https://www.dropbox.com/s/bavjtyx0ndty7xt/pretrain.txt?dl=0

%cd ..

mkdir: cannot create directory ‘./data’: File exists
/content/drive/MyDrive/Colab Notebooks/transformers/data
/content/drive/MyDrive/Colab Notebooks/transformers


In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

# we can check the parameters of this tokenizer
tokenizer

BertTokenizer(name_or_path='bert-base-cased', vocab_size=28996, model_max_length=512, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

In [ ]:
#from transformers import AutoConfig

model = BertModel.from_pretrained("bert-base-cased")

#config = AutoConfig.from_pretrained("bert-base-cased")

#108 M
print(f"Model size: {model.num_parameters()}")

#model summary
model

Model size: 108310272


BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(28996, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
  

### Pre-training BERT

Vamos a pre entrenar un MLM

#### Pre-training

#Preparando el dataset

Vamos a entrenar BERT para predecir X% de los tokens (15% en el paper original) de los cuales el 80% los reemplazamos por ``[MASK]``, 10% por un random token y 10% retenemos el mismo token.

In [ ]:
class PretrainDataset(torch.utils.data.Dataset):

    def __init__(self, tokenizer, input_file):

        self.tokenizer = tokenizer

        self.texts = self.read_text(input_file)

        self.mlm_probability = 0.15

    def read_text(self, input_file):

        fin = open(input_file)
        return fin.readlines()

    def collate_fn(self, batch):

        batch = self.tokenizer(batch, return_tensors='pt', padding=True, truncation=True, max_length=128)

        inputs, labels = self.mask_tokens(batch["input_ids"])
        return {"input_ids": inputs, "labels": labels}

        return encodings

    def mask_tokens(self, inputs):
        """
        Prepare masked tokens inputs/labels for masked language modeling: 80% MASK, 10% random, 10% original.
        """
        if self.tokenizer.mask_token is None:
            raise ValueError(
                "This tokenizer does not have a mask token which is necessary for masked language modeling. Remove the --mlm flag if you want to use this tokenizer."
            )
        labels = inputs.clone()

        # We sample a few tokens in each sequence for masked-LM training (with probability args.mlm_probability defaults to 0.15 in Bert/RoBERTa)
        probability_matrix = torch.full(labels.shape, self.mlm_probability)
        special_tokens_mask = [
            self.tokenizer.get_special_tokens_mask(val, already_has_special_tokens=True) for val in labels.tolist()
        ]
        probability_matrix.masked_fill_(torch.tensor(special_tokens_mask, dtype=torch.bool), value=0.0)

        if self.tokenizer._pad_token is not None:
            padding_mask = labels.eq(self.tokenizer.pad_token_id)
            probability_matrix.masked_fill_(padding_mask, value=0.0)
        masked_indices = torch.bernoulli(probability_matrix).bool()
        labels[~masked_indices] = -100  # We only compute loss on masked tokens

        # 80% of the time, we replace masked input tokens with tokenizer.mask_token ([MASK])
        indices_replaced = torch.bernoulli(torch.full(labels.shape, 0.8)).bool() & masked_indices
        inputs[indices_replaced] = self.tokenizer.convert_tokens_to_ids(self.tokenizer.mask_token)

        # 10% of the time, we replace masked input tokens with random word
        indices_random = torch.bernoulli(torch.full(labels.shape, 0.5)).bool() & masked_indices & ~indices_replaced
        random_words = torch.randint(len(self.tokenizer), labels.shape, dtype=torch.long)
        inputs[indices_random] = random_words[indices_random]

        # The rest of the time (10% of the time) we keep the masked input tokens unchanged
        return inputs, labels

    def __len__(self):

        return len(self.texts)

    def __getitem__(self, idx):

        text = self.texts[idx]
        return text

Usanos el MLM head de HF

In [ ]:
from transformers.models.bert.modeling_bert import BertOnlyMLMHead

class BERT_pretrain(BertPreTrainedModel):

    def __init__(self, config):
        super().__init__(config)
        self.config = config

        self.bert = BertModel(config)
        self.cls = BertOnlyMLMHead(config)

        self.init_weights()

    def forward(
        self,
        input_ids=None,
        attention_mask=None,
        token_type_ids=None,
        position_ids=None,
        head_mask=None,
        inputs_embeds=None,
        labels=None,
        output_attentions=None,
        output_hidden_states=None,
        return_dict=None):

        outputs = self.bert(
            input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            position_ids=position_ids,
            head_mask=head_mask,
            inputs_embeds=inputs_embeds,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
        )

        prediction_scores = self.cls(outputs[0])
        return prediction_scores

In [ ]:
class Trainer_MLM(Trainer):
    def compute_loss(self, model, inputs):

        labels = inputs['labels']

        outputs = model(**inputs)

        # MLM loss
        lm_loss = nn.CrossEntropyLoss()

        loss_mlm = lm_loss(outputs.view(-1, model.config.vocab_size), labels.view(-1))

        loss = loss_mlm

        return loss

In [ ]:
def main_pretrain():

    model = BERT_pretrain.from_pretrained("bert-base-cased")

    tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
    pretrain_dataset = PretrainDataset(tokenizer, 'data/pretrain.txt')

    training_args = TrainingArguments(
        output_dir='./experiment/pretrain',
        learning_rate = 0.00005,
        num_train_epochs =1,
        save_steps = 10000,  #saves a checkpoint file every 10000 iterations
        per_device_train_batch_size=64,
        remove_unused_columns=False
    )
    trainer = Trainer_MLM(
        model=model,
        args=training_args,
        train_dataset=pretrain_dataset,
        data_collator=pretrain_dataset.collate_fn
    )

    trainer.train()

    trainer.save_model('./models/ht_bert_pretrained/')


In [ ]:
 main_pretrain()

Some weights of BERT_pretrain were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['cls.predictions.decoder.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.


Step,Training Loss
500,5.895800
1000,4.770900
1500,4.440900
